In [1]:
import pandas as pd
import sqlite3


excel_file = "class_11.xlsx"  
df = pd.read_excel(excel_file)

df.columns = df.columns.str.lower()

df.head()

,id,title,authors,translators,subjects,bookshelves,languages,copyright,media_type,formats,download_count
0,84,"Frankenstein; Or, The Modern Prometheus","Shelley, Mary Wollstonecraft",NaN,Frankenstein's monster (Fictitious character) ...,Culture/Civilization/Society; Fiction,en,False,Text,text/html,78467
1,1513,Romeo and Juliet,"Shakespeare, William",NaN,Conflict of generations -- Drama; Juliet (Fict...,Fiction; Literature,en,False,Text,text/html,62300
2,1342,Pride and Prejudice,"Austen, Jane",NaN,Courtship -- Fiction; Domestic fiction; Englan...,Best Books Ever Listings; Culture/Civilization...,en,False,Text,text/html,58812
3,2701,"Moby Dick; Or, The Whale","Melville, Herman",NaN,"Adventure stories; Ahab, Captain (Fictitious c...",Best Books Ever Listings; Fiction,en,False,Text,text/html,58362
4,25344,The Scarlet Letter,"Hawthorne, Nathaniel",NaN,Adultery -- Fiction; Boston (Mass.) -- History...,Banned Books from Anne Haight's list; Culture/...,en,False,Text,text/html,45002


In [ ]:
# import sqlite3
# import pandas as pd
import re

# Подключаемся к базе и загружаем данные
conn = sqlite3.connect("gutenberg_books.db")
df = pd.read_sql_query("SELECT * FROM books", conn)

def format_authors(authors_str):
    """
    Преобразует строку с авторами в формат: Имя Фамилия
    """
    if pd.isna(authors_str):
        return None  # если поле пустое

    # Разделяем авторов по ";"
    authors = [a.strip() for a in authors_str.split(';')]

    formatted_authors = []
    for author in authors:
        if ',' in author:
            # Разбиваем по запятой на фамилию и имя
            surname, name = author.split(',', 1)
            name = name.strip()
            surname = surname.strip()

            # Убираем скобки, если есть (например, "T. (Tobias)")
            match = re.search(r'\((.*?)\)', name)
            if match:
                name = match.group(1)  # Берём полное имя из скобок
            else:
                # Если нет скобок, используем как есть
                name = name

            formatted_authors.append(f"{name} {surname}")
        else:
            # Если нет запятой, оставляем как есть
            formatted_authors.append(author.strip())

    return ', '.join(formatted_authors)

# Применяем функцию к столбцу author
df['authors'] = df['authors'].apply(format_authors)

# Проверим результат
print(df[['title', 'authors']].head())

# Записываем обратно в таблицу
df.to_sql("books", conn, if_exists="replace", index=False)
conn.close()

print("✅ Столбец 'authors' успешно отформатирован и обновлён в таблице!")


                                       title                      authors
0           Alice's Adventures in Wonderland                Lewis Carroll
1    Frankenstein; Or, The Modern Prometheus  Mary Wollstonecraft Shelley
2  The Complete Works of William Shakespeare          William Shakespeare
3                                Middlemarch                 George Eliot
4                 The Picture of Dorian Gray                  Oscar Wilde
✅ Столбец 'authors' успешно отформатирован и обновлён в таблице!


In [4]:
conn = sqlite3.connect("gutenberg_books.db")  # если файла нет, он создастся автоматически
cursor = conn.cursor()


cursor.execute("DROP TABLE IF EXISTS books")

# ✅ Теперь создаём её заново
cursor.execute('''
CREATE TABLE books (
    id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    authors TEXT,
    translators TEXT,
    subjects TEXT,
    bookshelves TEXT,
    languages TEXT,
    copyright BOOLEAN,
    media_type TEXT,
    formats TEXT,
    download_count INTEGER
)
''')
print("✅ Таблица создана или уже существует.")


# Переименуем колонки, чтобы совпадали с таблицей
# df.rename(columns={
#     'ID': 'id',
#     'TITLE': 'title',
#     'AUTHORS': 'author',
#     'TRANSLATORS': 'translator',
#     'SUBJECTS': 'subjects',
#     'BOOKSHELVES': 'bookshelves',
#     'LANGUAGES': 'language',
#     'COPYRIGHT': 'copyright',
#     'MEDIA_TYPE': 'media_type',
#     'FORMATS': 'formats',
#     'DOWNLOAD_COUNT': 'download_count'
# }, inplace=True)


# ✅ Загружаем данные в таблицу
df.to_sql("books", conn, if_exists="append", index=False)
print("✅ Данные успешно загружены в таблицу.")

# # ✅ Закрываем подключение
# conn.commit()
# conn.close()
# print("✅ Готово! База данных создана: gutenberg_books.db")

✅ Таблица создана или уже существует.
✅ Данные успешно загружены в таблицу.


In [ ]:
# import sqlite3
# import pandas as pd
# import re

# # Подключаемся к базе и загружаем данные
# conn = sqlite3.connect("gutenberg_books.db")
# df = pd.read_sql_query("SELECT * FROM books", conn)

# def format_authors(authors_str):
#     """
#     Преобразует строку с авторами в формат: Имя Фамилия
#     """
#     if pd.isna(authors_str):
#         return None  # если поле пустое

#     # Разделяем авторов по ";"
#     authors = [a.strip() for a in authors_str.split(';')]
#     # print("📖 authors =", authors)  # выводим список авторов

#     # пока возвращаем список авторов вместо обработки
# #     return authors

# # print(format_authors("Shakespeare, William"))


#     formatted_authors = []
#     for author in authors:
#         if ',' in author:
#             # Разбиваем по запятой на фамилию и имя
#             surname, name = author.split(',', 1)
#             name = name.strip()
#             surname = surname.strip()

#             # Убираем скобки, если есть (например, "T. (Tobias)")
#             match = re.search(r'\((.*?)\)', name)
#             if match:
#                 name = match.group(1)  # Берём полное имя из скобок
#             else:
#                 # Если нет скобок, используем как есть
#                 name = name

#             formatted_authors.append(f"{name} {surname}")
#         else:
#             # Если нет запятой, оставляем как есть
#             formatted_authors.append(author.strip())

#     return ', '.join(formatted_authors)

# # Применяем функцию к столбцу author
# df['authors'] = df['authors'].apply(format_authors)

# # Проверим результат
# print(df[['title', 'authors']].head())

# # Записываем обратно в таблицу
# df.to_sql("books", conn, if_exists="replace", index=False)
# conn.close()

# print("✅ Столбец 'authors' успешно отформатирован и обновлён в таблице!")


['Shakespeare, William']


In [ ]:
# mystr = "один два три четыре пять"
# str_list = mystr.split(" ", 3)
# print(str_list) 

['один', 'два', 'три', 'четыре пять']


In [ ]:
import requests
import pandas as pd
import sqlite3
import re


def fetch_books(language):
    url = f"https://gutendex.com/books?languages={language}"
    response = requests.get(url)
    response.raise_for_status()  
    data = response.json()
    return data['results']


In [ ]:
# def process_name(author):
#     """
#     Переводит имя в формат 'Имя Фамилия'
#     """
#     if ',' in author['name']:
#         surname, name = author['name'].split(',', 1)
#         return name.strip() + ' ' + surname.strip()
#     return author['name'].strip()


def converse_name(author_dict):
    
    # Фамилия, Имя -> Имя Фамилия

    name_str = author_dict.get('name', '').strip()

    if not name_str:
        return ''  

    if ',' in name_str:
        # Разбиваем на Фамилию и Имя
        surname, name = name_str.split(',', 1)
        name = name.strip()
        surname = surname.strip()

        # Убираем скобки, если есть (например, "E. M. (Edward Morgan)" -> "Edward")
        match = re.search(r'\((.*?)\)', name)
        if match:
            name = match.group(1)

        return f"{name} {surname}"
    else:
        return name_str  


def process_book(book):
    
    return {
        'id': book['id'],
        'title': book['title'],
        'copyright': book['copyright'],
        'media_type': book['media_type'],
        'download_count': book['download_count'],
        'authors': '; '.join([converse_name(a) for a in book['authors']]),
        'translators': '; '.join([converse_name(t) for t in book['translators']]),
        'subjects': '; '.join(book['subjects'][:4]),  
        'bookshelves': '; '.join(
            [re.sub(r'Browsing:\s*', '', shelf) for shelf in book['bookshelves'][:2]]
        ),  
        'languages': book['languages'][0] if book['languages'] else None,
        'formats': list(book['formats'].keys())[0] if book['formats'] else None
    }


books_all = []
for lang in ['en', 'es', 'fr']:
    books = fetch_books(lang)
    processed = [process_book(b) for b in books]
    books_all.extend(processed)


df = pd.DataFrame(books_all)

conn = sqlite3.connect("gutendex_books.db")
df.to_sql("books", conn, if_exists="replace", index=False)
conn.close()

print("Data successfully written to gutendex_books.db")

✅ Данные успешно записаны в gutendex_books.db


In [3]:
df.head()

,id,title,copyright,media_type,download_count,authors,translators,subjects,bookshelves,languages,formats
0,2701,"Moby Dick; Or, The Whale",False,Text,104393,Herman Melville,,"Adventure stories; Ahab, Captain (Fictitious c...",Best Books Ever Listings; Fiction,en,text/html
1,84,"Frankenstein; Or, The Modern Prometheus",False,Text,89216,Mary Wollstonecraft Shelley,,Frankenstein's monster (Fictitious character) ...,Culture/Civilization/Society; Fiction,en,text/html
2,7142,The History of the Peloponnesian War,False,Text,67806,Thucydides,Richard Crawley,"Greece -- History -- Peloponnesian War, 431-40...",History - European; History - General,en,text/html
3,1513,Romeo and Juliet,False,Text,66718,William Shakespeare,,Conflict of generations -- Drama; Juliet (Fict...,Fiction; Literature,en,text/html
4,2641,A Room with a View,False,Text,61156,Edward Morgan Forster,,British -- Italy -- Fiction; England -- Fictio...,Culture/Civilization/Society; Fiction,en,text/html


In [4]:
# import sqlite3
# import pandas as pd
# import re

# Подключаемся к базе и загружаем данные
# conn = sqlite3.connect("gutendex_books.db")
# df = pd.read_sql_query("SELECT * FROM books", conn)

# def format_authors(authors_str):
#     """
#     Преобразует строку с авторами в формат: Имя Фамилия
#     """
#     if pd.isna(authors_str):
#         return None  # если поле пустое

#     # Разделяем авторов по ";"
#     authors = [a.strip() for a in authors_str.split(';')]

#     formatted_authors = []
#     for author in authors:
#         if ',' in author:
#             # Разбиваем по запятой на фамилию и имя
#             name, surname = author.split(',', 1)
#             name = name.strip()
#             surname = surname.strip()

#             # Убираем скобки, если есть (например, "T. (Tobias)")
#             match = re.search(r'\((.*?)\)', name)
#             if match:
#                 name = match.group(1)  # Берём полное имя из скобок
#             else:
#                 # Если нет скобок, используем как есть
#                 name = name

#             formatted_authors.append(f"{name} {surname}")
#         else:
#             # Если нет запятой, оставляем как есть
#             formatted_authors.append(author.strip())

#     return ', '.join(formatted_authors)


# conn = sqlite3.connect("gutendex_books.db")
# df = pd.read_sql_query("SELECT * FROM books", conn)

# # Применяем функцию к каждому автору
# df['authors'] = df['authors'].apply(
#     lambda x: '; '.join([format_authors(a) for a in x.split(';')]) if pd.notnull(x) else None
# )

# # Проверим результат
# print(df[['title', 'authors']].head())

# # Записываем обратно в таблицу
# df.to_sql("books", conn, if_exists="replace", index=False)
# conn.close()

# print("✅ Столбец 'authors' успешно отформатирован и обновлён в таблице!")
